# Hospital AI Chatbot Backend - Professional Implementation

This notebook implements a sophisticated AI-powered chatbot backend for hospital information systems. The system integrates advanced NLP techniques, machine learning, and reinforcement learning to provide intelligent responses about medical services, appointments, and hospital information.

## Key Features:
- **Advanced NLP**: Sentence Transformers for semantic understanding
- **Machine Learning**: TF-IDF vectorization with cosine similarity
- **Reinforcement Learning**: User feedback integration for continuous improvement
- **Multi-modal Architecture**: Context-based + AI model responses
- **Production Ready**: Flask API with comprehensive endpoints

## Import Required Libraries

Loading all necessary dependencies for NLP, machine learning, web framework, and data processing.

In [ ]:
import csv
import os
import re
import threading
import time
import json
import numpy as np
from datetime import datetime
from flask import Flask, request, jsonify

# Initialize Flask app
app = Flask(__name__)

print("✓ Core libraries imported successfully")

## NLPEnhancedChatbot Class - Core Implementation

The main chatbot class implementing advanced NLP techniques, machine learning algorithms, and reinforcement learning for intelligent medical information assistance.

In [ ]:
class NLPEnhancedChatbot:
    def __init__(self, csv_file_path=None):
        self.qa_pairs = []
        self.original_qa_count = 0  # Track original count for learning statistics
        self.context_data = ""
        self.model = None
        self.tokenizer = None
        self.model_type = "Loading..."
        self.model_loading = True
        
        # ML/RL components
        self.conversation_history = []
        self.feedback_scores = {}
        self.response_quality = {}
        self.learning_rate = 0.1
        self.vectorizer = None
        self.qa_vectors = None
        
        # Advanced NLP components
        self.sentence_transformer = None
        self.nlp_processor = None
        self.intent_classifier = None
        self.context_embeddings = None
        self.domain_knowledge = {}
        
        # Load Q&A data if available
        if csv_file_path and os.path.exists(csv_file_path):
            self.load_qa_data(csv_file_path)
        
        # Initialize NLP components
        self.initialize_nlp_components()
        
        # Start model loading in background thread
        self.model_loading_thread = threading.Thread(target=self.initialize_model, daemon=True)
        self.model_loading_thread.start()
        
        print("✓ NLPEnhancedChatbot initialized successfully")

## Advanced NLP Components Initialization

Setting up sophisticated NLP components including Sentence Transformers, intent classification, and semantic understanding capabilities.

In [ ]:
    def initialize_nlp_components(self):
        """Initialize advanced NLP components for semantic understanding"""
        try:
            print("Initializing advanced NLP components...")
            
            # Initialize sentence transformer for semantic embeddings
            self.initialize_sentence_transformer()
            
            # Initialize intent classification
            self.initialize_intent_classifier()
            
            # Build domain knowledge from Q&A pairs
            self.build_domain_knowledge()
            
            # Create semantic embeddings for existing data
            if self.qa_pairs:
                self.create_semantic_embeddings()
            
            print("✓ Advanced NLP components initialized successfully")
            
        except Exception as e:
            print(f"Error initializing NLP components: {e}")
            # Fallback to basic TF-IDF
            self.initialize_fallback_ml()
    
    def initialize_sentence_transformer(self):
        """Initialize sentence transformer for semantic understanding"""
        try:
            from sentence_transformers import SentenceTransformer
            
            # Use a lightweight but effective model
            model_name = 'all-MiniLM-L6-v2'
            print(f"Loading sentence transformer: {model_name}")
            self.sentence_transformer = SentenceTransformer(model_name)
            print("✓ Sentence transformer loaded successfully")
            
        except ImportError:
            print("⚠ sentence-transformers not available, using fallback")
            self.sentence_transformer = None
        except Exception as e:
            print(f"Error loading sentence transformer: {e}")
            self.sentence_transformer = None

## Intent Classification System

Implementing intelligent intent recognition for medical queries including appointments, pricing, emergency contacts, and hospital information.

In [ ]:
    def initialize_intent_classifier(self):
        """Initialize intent classification for understanding user queries"""
        try:
            # Define common healthcare intents
            self.intent_patterns = {
                'appointment': ['appointment', 'book', 'schedule', 'visit', 'consultation', 'see doctor'],
                'pricing': ['price', 'cost', 'how much', 'expensive', 'fee', 'charge', 'bill'],
                'hospital_info': ['hours', 'visiting', 'location', 'address', 'directions', 'parking'],
                'emergency': ['emergency', 'urgent', 'ambulance', '911', 'critical', 'accident'],
                'departments': ['department', 'specialist', 'doctor', 'cardiology', 'neurology', 'oncology'],
                'insurance': ['insurance', 'cover', 'nhif', 'payment', 'billing', 'claim'],
                'medical_records': ['records', 'results', 'report', 'test', 'x-ray', 'lab'],
                'symptoms': ['pain', 'fever', 'headache', 'chest', 'stomach', 'symptoms'],
                'pharmacy': ['medicine', 'prescription', 'drug', 'pharmacy', 'medication'],
                'general': ['hello', 'hi', 'greetings', 'thank you', 'bye', 'goodbye', 'help']
            }
            
            print("✓ Intent classification patterns loaded")
            
        except Exception as e:
            print(f"Error initializing intent classifier: {e}")
    
    def classify_intent(self, user_input):
        """Classify the intent of the user input"""
        user_input_lower = user_input.lower()
        intent_scores = {}
        
        for intent, patterns in self.intent_patterns.items():
            score = 0
            for pattern in patterns:
                if pattern in user_input_lower:
                    score += 1
            intent_scores[intent] = score
        
        # Return the intent with highest score, or 'general' if no clear intent
        if max(intent_scores.values()) > 0:
            return max(intent_scores, key=intent_scores.get)
        return 'general'

## Data Loading and Vectorization

Loading hospital Q&A data and creating machine learning vectors for semantic similarity matching.

In [ ]:
    def load_qa_data(self, file_path):
        """Load questions and answers from CSV file and create ML vectors"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    # Handle both old format (Question/Answer) and new format (question/answer)
                    question = row.get("question", row.get("Question", "")).strip().strip('"')
                    answer = row.get("answer", row.get("Answer", "")).strip().strip('"')
                    category = row.get("category", "general")
                    hospital = row.get("hospital", "both")
                    
                    self.qa_pairs.append({
                        "question": question, 
                        "answer": answer,
                        "category": category,
                        "hospital": hospital
                    })
                    # Add to context for the model
                    self.context_data += f"Q: {question}\nA: {answer}\n\n"
            
            print(f"Loaded {len(self.qa_pairs)} medical Q&A pairs from hospital dataset")
            
            # Set original count for learning statistics
            self.original_qa_count = len(self.qa_pairs)
            
            # Create TF-IDF vectors for ML-enhanced matching
            self.create_qa_vectors()
            
        except Exception as e:
            print(f"Error loading Q&A data: {e}")
            
    def create_qa_vectors(self):
        """Create TF-IDF vectors for all Q&A pairs for semantic matching"""
        try:
            if self.vectorizer and self.qa_pairs:
                questions = [qa["question"] for qa in self.qa_pairs]
                self.qa_vectors = self.vectorizer.fit_transform(questions)
                print("Created TF-IDF vectors for semantic matching")
        except Exception as e:
            print(f"Could not create vectors: {e}")

## Semantic Understanding with Sentence Transformers

Advanced semantic similarity matching using state-of-the-art sentence transformers for better understanding of medical queries.

In [ ]:
    def create_semantic_embeddings(self):
        """Create semantic embeddings for all Q&A pairs"""
        try:
            if not self.sentence_transformer:
                return
                
            # Create embeddings for questions and answers
            questions = [qa['question'] for qa in self.qa_pairs]
            answers = [qa['answer'] for qa in self.qa_pairs]
            
            print("Creating semantic embeddings...")
            self.question_embeddings = self.sentence_transformer.encode(questions)
            self.answer_embeddings = self.sentence_transformer.encode(answers)
            
            print(f"✓ Created embeddings for {len(questions)} Q&A pairs")
            
        except Exception as e:
            print(f"Error creating semantic embeddings: {e}")
    
    def get_semantic_answer(self, user_input, intent):
        """Get answer using semantic similarity with sentence transformers"""
        try:
            if not self.sentence_transformer or not hasattr(self, 'question_embeddings'):
                return None
            
            # Encode the user input
            user_embedding = self.sentence_transformer.encode([user_input])
            
            # Calculate semantic similarity with questions
            similarities = np.dot(user_embedding, self.question_embeddings.T).flatten()
            
            # Find best matches
            best_indices = np.argsort(similarities)[::-1][:3]
            best_scores = similarities[best_indices]
            
            # Check if best match is semantically similar (threshold of 0.3)
            if best_scores[0] > 0.3:
                best_match = self.qa_pairs[best_indices[0]]
                
                # Generate contextual response based on intent
                enhanced_answer = self.enhance_answer_with_intent(
                    best_match['answer'], intent, user_input
                )
                
                return {
                    'answer': enhanced_answer,
                    'confidence': float(best_scores[0]),
                    'source': 'semantic_nlp',
                    'intent': intent,
                    'method': 'sentence_transformer'
                }
            
            return None
            
        except Exception as e:
            print(f"Error in semantic answer generation: {e}")
            return None

## Dynamic Answer Generation

Intelligent response generation based on medical intents including appointments, pricing, emergency contacts, and hospital services.

In [ ]:
    def generate_dynamic_answer(self, user_input, intent):
        """Generate dynamic answers based on intent and medical domain knowledge"""
        
        # Extract key entities from user input
        entities = self.extract_entities(user_input)
        
        # Generate intent-specific responses
        if intent == 'appointment':
            return self.generate_appointment_response(entities, user_input)
        
        elif intent == 'pricing':
            return self.generate_pricing_response(entities, user_input)
        
        elif intent == 'hospital_info':
            return self.generate_hospital_info_response(entities, user_input)
        
        elif intent == 'emergency':
            return self.generate_emergency_response(entities, user_input)
        
        elif intent == 'departments':
            return self.generate_departments_response(entities, user_input)
        
        elif intent == 'insurance':
            return self.generate_insurance_response(user_input)
        
        elif intent == 'medical_records':
            return self.generate_medical_records_response(user_input)
        
        elif intent == 'symptoms':
            return self.generate_symptoms_response(entities, user_input)
        
        elif intent == 'pharmacy':
            return self.generate_pharmacy_response(user_input)
        
        elif intent == 'general':
            return self.generate_general_response(user_input)
        
        return None

## Medical Response Generators

Specialized response generators for different medical service categories providing accurate and helpful information.

In [ ]:
    def generate_appointment_response(self, entities, user_input):
        """Generate appointment booking responses"""
        return {
            'answer': "To book an appointment: Nairobi Hospital: +254-20-2845000 or online portal. Kenyatta National Hospital: +254-20-2726300. For specialists, please book 2-3 days in advance. Emergency services available 24/7.",
            'confidence': 0.8,
            'source': 'dynamic_generation',
            'intent': 'appointment',
            'method': 'template_based'
        }
    
    def generate_emergency_response(self, entities, user_input):
        """Generate emergency contact responses"""
        return {
            'answer': "EMERGENCY CONTACTS: Nairobi Hospital: +254-20-2845000 | Kenyatta National: +254-20-2726300 | Both hospitals operate 24/7 emergency services. For life-threatening situations, call immediately.",
            'confidence': 0.9,
            'source': 'dynamic_generation',
            'intent': 'emergency',
            'method': 'template_based'
        }
    
    def generate_pricing_response(self, entities, user_input):
        """Generate dynamic medical pricing responses"""
        return {
            'answer': "Medical service pricing varies by procedure and hospital. General ranges: CT scan 8,000-25,000 KSh, Normal delivery 25,000-120,000 KSh, C-section 60,000-200,000 KSh. Insurance coverage available. Contact billing for specific procedures.",
            'confidence': 0.7,
            'source': 'dynamic_generation',
            'intent': 'pricing',
            'method': 'template_based'
        }
    
    def generate_departments_response(self, entities, user_input):
        """Generate department information responses"""
        return {
            'answer': "Available departments: Cardiology, Neurology, Oncology, Pediatrics, Orthopedics, Radiology, Emergency Medicine, Maternity, Surgery, Internal Medicine, Psychiatry, Dermatology, and more. Specialist appointments available.",
            'confidence': 0.8,
            'source': 'dynamic_generation',
            'intent': 'departments',
            'method': 'template_based'
        }

## Machine Learning Fallback System

TF-IDF based similarity matching as a fallback when advanced NLP methods are not available or don't produce confident results.

In [ ]:
    def initialize_fallback_ml(self):
        """Fallback ML initialization if advanced NLP fails"""
        try:
            from sklearn.feature_extraction.text import TfidfVectorizer
            from sklearn.metrics.pairwise import cosine_similarity
            
            print("Initializing fallback ML components...")
            
            # Initialize TF-IDF vectorizer for semantic matching
            self.vectorizer = TfidfVectorizer(
                max_features=1000,
                stop_words='english',
                ngram_range=(1, 2)
            )
            
            # Load conversation history if exists
            self.load_conversation_history()
            
            # Load feedback scores if exists
            self.load_feedback_scores()
            
            print("ML components initialized successfully")
            
        except ImportError:
            print("WARNING: scikit-learn not available. Using basic matching.")
            self.vectorizer = None
    
    def get_ml_context_answer(self, user_input):
        """Fallback ML method for enhanced context matching"""
        if not self.qa_pairs or not self.vectorizer or self.qa_vectors is None:
            return self.get_basic_context_answer(user_input)
            
        try:
            from sklearn.metrics.pairwise import cosine_similarity
            
            # Vectorize user input
            user_vector = self.vectorizer.transform([user_input])
            
            # Calculate cosine similarity with all Q&A pairs
            similarities = cosine_similarity(user_vector, self.qa_vectors).flatten()
            
            # Find best matches
            best_indices = np.argsort(similarities)[::-1][:3]  # Top 3 matches
            best_scores = similarities[best_indices]
            
            # Check if best match is good enough (threshold of 0.1)
            if best_scores[0] > 0.1:
                best_match = self.qa_pairs[best_indices[0]]
                
                return {
                    'answer': best_match['answer'],
                    'confidence': min(best_scores[0] * 2, 1.0),  # Scale to 0-1
                    'source': 'ml_context',
                    'method': 'tfidf_cosine_similarity'
                }
                
        except ImportError:
            return self.get_basic_context_answer(user_input)
        except Exception as e:
            print(f"ML matching error: {e}")
            return self.get_basic_context_answer(user_input)
            
        return None

## Flask API Endpoints

Production-ready REST API endpoints for the hospital chatbot system with comprehensive functionality.

In [ ]:
# Initialize chatbot - make CSV path optional since we're using pretrained model
csv_path = "../data/hospital_comprehensive_data.csv"
if not os.path.exists(csv_path):
    print(f"Warning: CSV file not found at {csv_path}. Using pretrained model only.")
    csv_path = None

chatbot = NLPEnhancedChatbot(csv_path)

# Root route for basic info
@app.route('/', methods=['GET'])
def home():
    return jsonify({
        "message": "Hospital AI Agent Backend is running!",
        "endpoints": {
            "chat": "/chat (POST)",
            "health": "/health (GET)", 
            "stats": "/stats (GET)"
        },
        "qa_pairs_loaded": len(chatbot.qa_pairs)
    })

# API endpoint for chatbot
@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.json
        user_message = data.get("message", "").strip()
        
        if not user_message:
            return jsonify({"error": "No message provided"}), 400
        
        response_data = chatbot.get_response(user_message)
        
        return jsonify({
            "response": response_data["response"],
            "confidence": response_data["confidence"],
            "source": response_data["source"],
            "status": "success"
        })
    
    except Exception as e:
        return jsonify({
            "error": str(e),
            "status": "error"
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint with model loading status"""
    if chatbot.model_loading:
        model_status = f"Loading... ({chatbot.model_type})"
    elif hasattr(chatbot, 'model') and chatbot.model is not None:
        model_status = chatbot.model_type
    else:
        model_status = chatbot.model_type
    
    return jsonify({
        "status": "healthy",
        "qa_pairs_loaded": len(chatbot.qa_pairs),
        "model_type": model_status,
        "model_loading": chatbot.model_loading,
        "context_available": len(chatbot.context_data) > 0
    })

## Server Startup and Testing

Start the Flask development server for testing the chatbot API endpoints.

In [ ]:
# Test the chatbot initialization
if __name__ == "__main__":
    print("Hospital AI Chatbot Backend - Notebook Version")
    print(f"Loaded {len(chatbot.qa_pairs)} Q&A pairs as context")
    print("AI model loading in background...")
    print("Ready for testing!")
    
    # Test a sample query
    sample_response = chatbot.get_response("How do I book an appointment?")
    print("\nSample Query Test:")
    print(f"Response: {sample_response['response']}")
    print(f"Confidence: {sample_response['confidence']}")
    print(f"Source: {sample_response['source']}")

## Manual Testing Interface

Interactive testing cell for manually testing the chatbot responses.

In [ ]:
# Interactive testing function
def test_chatbot_query(query):
    """Test a specific query with the chatbot"""
    print(f"Query: {query}")
    response = chatbot.get_response(query)
    print(f"Response: {response['response']}")
    print(f"Confidence: {response['confidence']:.3f}")
    print(f"Source: {response['source']}")
    print("-" * 50)

# Test various medical queries
test_queries = [
    "What are the emergency contact numbers?",
    "How much does a CT scan cost?",
    "What departments are available?",
    "How do I get my medical records?",
    "Hello, I need help"
]

print("Testing Hospital AI Chatbot:")
print("=" * 50)
for query in test_queries:
    test_chatbot_query(query)